In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.chrome.options import Options
import os
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException
from IPython.display import display, clear_output
import ipywidgets as widgets
import tkinter as tk
from tkinter import filedialog
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton
import sys
import re
import zipfile #extract zip file
import requests
from bs4 import BeautifulSoup
PATH = "/Users/frrookie521/Desktop/civil/cee495/cee495/Google\ Chrome\ for\ Testing.app"
download_path = "//Users/frrookie521/Desktop/civil/cee495/2022"
chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
prefs = {
        'download.default_directory': download_path,
        'download.prompt_for_download': False,
        'download.directory_upgrade': True,
        'safebrowsing.enabled': True
         }
chrome_options.add_experimental_option('prefs', prefs)
excel_file = pd.read_excel('VariableSources_2.xlsx', header=None)
driver = webdriver.Chrome(options=chrome_options)
driver.set_window_size(1396, 1000)
urls = excel_file.iloc[1:, 2].tolist() #store urls from third column and second row 
unique = set(url for url in urls if isinstance(url, str)) #extact the unique urls from excel
zipfiles = ['file1.zip', 'file2.zip'] 
def census_construction(driver,unique):
    base = 'https://www2.census.gov/econ/bps/County/'
    driver.get(base)
    time.sleep(5)
    def process_txt_content(txt_content, output_csv_file):
        lines = txt_content.split('\n')
        with open(output_csv_file, 'w') as csv_file:
            for line in lines:
                fields = line.strip().split()  # Adjust if needed
                csv_line = ','.join(fields)
                csv_file.write(csv_line + '\n')
    fourth_digit = 2
    third_digit = 2   #from year 2018 
    stop_loop = False #we define the stop loop for the entire iteration 
    generated_filenames = set()
    while fourth_digit < 10 and not stop_loop:
        for last_two_digits in range(1, 14):  # The last two digits are changing from 1 to 13
            if fourth_digit == 2 and third_digit == 2 and last_two_digits == 12:
                if file_type == 'c':
                    stop_loop = True
                    driver.quit()
                    break 
            if last_two_digits > 12:
                third_digit += 1
                if third_digit == 10:
                    fourth_digit += 1
                    third_digit = 0
                adjusted_last_digits = last_two_digits - 12 
                adjusted = True
            else:
                adjusted_last_digits = last_two_digits
                adjusted = False  
            for file_type in ['c', 'y']:
                if fourth_digit == 2 and third_digit == 3 and last_two_digits == 1 and file_type == 'c':
                    stop_loop = True
                    driver.quit()
                    break
                else:
                    file_name = f"co{str(fourth_digit)}{str(third_digit)}{str(adjusted_last_digits).zfill(2)}{file_type}.txt"
                    if file_name in generated_filenames:
                        continue
                    generated_filenames.add(file_name) 
                    full_url = base + file_name
                    driver.get(full_url)
                    driver.refresh()
                    time.sleep(2)
                    response = requests.get(full_url)
                    if response.status_code == 200:
                        content = response.text
                        local_file_path = f"co{str(fourth_digit)}{str(third_digit)}{str(adjusted_last_digits).zfill(2)}{file_type}.txt"
                        with open(local_file_path, "w") as file:
                            file.write(content)
                        table = pd.read_csv(local_file_path, sep=',')
                        table.reset_index(inplace=True)
                        table.drop(columns=['FIPS.1', 'Region', 'Unnamed: 6', 'Unnamed: 9', 'Unnamed: 12', 'Unnamed: 15', 'Unnamed: 17', 'Unnamed: 18', '1-unit rep', 'Unnamed: 20', 'Unnamed: 21', '2-units rep', 'Unnamed: 23', 'Unnamed: 24', '3-4 units rep', 'Unnamed: 26', 'Unnamed: 27', ' 5+units rep'], inplace=True)
                        table.rename(columns={'Survey': 'State', 'FIPS': 'County', 'Division': 'County Name', 'County': 'total1Unit', '1-unit': 'value1Unit', 'Unnamed: 8': 'total2Unit', '2-units': 'value2Unit', 'Unnamed: 11': 'total34Unit', '3-4 units': 'value34Unit', 'Unnamed: 14': 'total5Unit', '5+ units': 'value5Unit'}, inplace=True)
                        table.drop(index=0, inplace=True)
                        table['total1Unit'] = table['total1Unit'].astype(float)
                        table['total2Unit'] = table['total2Unit'].astype(float)
                        table['total34Unit'] = table['total34Unit'].astype(float)
                        table['total5Unit'] = table['total5Unit'].astype(float)
                        table['value1Unit'] = table['value1Unit'].astype(float)
                        table['value2Unit'] = table['value2Unit'].astype(float)
                        table['value34Unit'] = table['value34Unit'].astype(float)
                        table['value5Unit'] = table['value5Unit'].astype(float)
                        table[f'totalBuildings{fourth_digit}{third_digit}'] = table['total1Unit'] + table['total2Unit'] + table['total34Unit'] + table['total5Unit']
                        table[f'totalBuildingValue{fourth_digit}{third_digit}'] = table['value1Unit'] + table['value2Unit'] + table['value34Unit'] + table['value5Unit']
                        table['GEO_ID'] = "0500000US" + table['State'] + table['County']
                        formatted_columns = ['State', 'County']  # Add more column names as needed
                        for column in formatted_columns:
                            if column == 'State':
                                table[column] = table[column].apply(lambda x: f"{int(x):02}" if x.isdigit() else x)
                            elif column == 'County':
                                table[column] = table[column].apply(lambda x: f"{int(x):03}" if x.isdigit() else x)
                        columns = ['index', 'State', 'County Name','County',f'totalBuildings{fourth_digit}{third_digit}',f'totalBuildingValue{fourth_digit}{third_digit}','GEO_ID']
                        selected = table[columns]
                        csv_file = f"co{str(fourth_digit)}{str(third_digit)}{str(adjusted_last_digits).zfill(2)}{file_type}.csv"
                        selected.to_csv(csv_file, index=False)
                        os.remove(local_file_path)
                    else:
                        print("Failed to download the text file")    
                    if stop_loop:
                        break     
                if adjusted:
                    break
target = {
        "https://www.census.gov/construction/bps/index.html":census_construction
}
            
def check_target(unique):
    target_formats = list(target.keys()) #we save the notation for each target format 
    return any(url.startswith(format) for format in target_formats) #checking whether each url is matched with our target urls under each notation 

for url in unique:
    if check_target(unique):
        interaction_function = target[url] #get the corresponding interaction function
        interaction_function(driver, url)  
driver.quit()  

The chromedriver version (116.0.5845.96) detected in PATH at /opt/homebrew/bin/chromedriver might not be compatible with the detected chrome version (117.0.5938.92); currently, chromedriver 117.0.5938.92 is recommended for chrome 117.*, so it is advised to delete the driver in PATH and retry


MaxRetryError: HTTPConnectionPool(host='localhost', port=57274): Max retries exceeded with url: /session/3db1d913131af1e8036f089746d0cbb4/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f935b972130>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [3]:
import os
for filename in os.listdir('.'):
    if filename.endswith('c.csv'):
        os.remove(filename)


In [9]:
import pandas as pd
import glob
merged_table = pd.DataFrame()
co22_files = glob.glob('co22*.csv')
print(co22_files)
for file in co22_files:
    table = pd.read_csv(file)
    merged_table = pd.concat([merged_table, table], ignore_index=True)
    merged_table = merged_table.groupby('GEO_ID').sum().reset_index()
columns_to_drop = ['index', 'State','County']
merged_table = merged_table.drop(columns_to_drop, axis=1)
merged_table.to_csv('construction_22.csv', index=False)
table = pd.read_csv('construction_22.csv')
for filename in os.listdir('.'):
    if filename.endswith('y.csv'):
        os.remove(filename)
table

['co2202y.csv', 'co2203y.csv', 'co2208y.csv', 'co2212y.csv', 'co2204y.csv', 'co2205y.csv', 'co2209y.csv', 'co2206y.csv', 'co2210y.csv', 'co2211y.csv', 'co2207y.csv', 'co2201y.csv']


/var/folders/1n/7d_rynhd4xqg_dytyg39s0vm0000gn/T/ipykernel_28984/1184327406.py:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  merged_table = merged_table.groupby('GEO_ID').sum().reset_index()
/var/folders/1n/7d_rynhd4xqg_dytyg39s0vm0000gn/T/ipykernel_28984/1184327406.py:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  merged_table = merged_table.groupby('GEO_ID').sum().reset_index()
/var/folders/1n/7d_rynhd4xqg_dytyg39s0vm0000gn/T/ipykernel_28984/1184327406.py:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. E

,GEO_ID,totalBuildings22,totalBuildingValue22
0,0500000US01001,1716.0,5.600560e+08
1,0500000US01003,21649.0,6.247434e+09
2,0500000US01005,136.0,5.425120e+07
3,0500000US01007,71.0,2.577500e+07
4,0500000US01009,135.0,2.431849e+07
...,...,...,...
3028,0500000US56037,225.0,7.902493e+07
3029,0500000US56039,1471.0,3.690279e+09
3030,0500000US56041,346.0,9.390034e+07
3031,0500000US56043,15.0,2.907500e+06
